In [2]:
library(dplyr)
library(httr)
library(ggplot2)
library(getPass)
library(repr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [3]:
token = getPass()


 ········································


In [4]:
url_base = "https://bigdata-api.fiocruz.br"

endpoint = paste0(url_base,"/","show_tables")
request <- POST(url = endpoint,
          body = list("token" = token),
          encode = "json")

as_tibble(content(request))

databases
<list>
datasus-sim
datasus-sinasc
oobr-sindrome-gripal-exploracao
datasus-sim_preliminar
datasus-sinasc_preliminar
datasus-sih
datasus-sim-el
oobr-sindrome-gripal-semanal
datasus-cnes


In [18]:
convertRequestToDF <- function(request){
    variables = unlist(content(request)$columns)
    variables = variables[names(variables) == "name"]
    column_names <- unname(variables)
    values = content(request)$rows
    df <- as.data.frame(do.call(rbind,lapply(values,function(r) {
        row <- r
        row[sapply(row, is.null)] <- NA
        rbind(unlist(row))
    } )))
    names(df) <- column_names
    return(df)
}

In [6]:
#numero de nascimentos
estados <- c('RO','AC','AM','RR','PA','AP','TO','MA','PI','CE','RN','PB','PE','AL','SE','BA','MG','ES','RJ','SP','PR','SC','RS','MS','MT','GO','DF')
dataframe <- data.frame()

endpoint <- paste0(url_base,"/","sql_query")

for (estado in estados){
    # Nascimentos total por município

    params = paste0('{
      "token": {
        "token": "',token,'"
      },
      "sql": {
        "sql": {"query":"SELECT res_SIGLA_UF, ano_nasc, CODMUNNASC, COUNT(1) FROM \\"datasus-sinasc\\" WHERE res_SIGLA_UF = \'',estado,'\' GROUP BY res_SIGLA_UF, ano_nasc, CODMUNNASC", "fetch_size": 65000}
      }
    }')

    request <- POST(url = endpoint, body = params, encode = "form")
    df_mun <- convertRequestToDF(request)
    #names(df_mun) <- c('UF', 'Ano', 'Mes', 'Nascimentos')
    dataframe <- rbind(dataframe, df_mun)
    
    repeat {
        
        cursor <- content(request)$cursor
        
        params = paste0('{
          "token": {
            "token": "',token,'"
          },
          "sql": {
            "sql": {"query":"SELECT res_SIGLA_UF, ano_nasc, CODMUNNASC, COUNT(1) FROM \\"datasus-sinasc\\" WHERE res_SIGLA_UF = \'',estado,'\' GROUP BY res_SIGLA_UF, ano_nasc, CODMUNNASC", "fetch_size": 65000, "cursor": "',cursor,'"}
          }
        }')


        request <- POST(url = endpoint, body = params, encode = "form")
        
        if (length(content(request)$rows) == 0)
            break
        else print("oi")
        
        df_mun <- convertRequestToDF(request)
       # names(df_mun) <- c('UF', 'Ano', 'Mes', 'Nascimentos')
        dataframe <- rbind(dataframe, df_mun)
    }
}

In [7]:
head(dataframe)
dim(dataframe)

,res_SIGLA_UF,ano_nasc,CODMUNNASC,COUNT(1)
,<fct>,<fct>,<fct>,<fct>
1,RO,1996,1100000,1
2,RO,1996,1100015,609
3,RO,1996,1100023,2611
4,RO,1996,1100031,137
5,RO,1996,1100049,2564
6,RO,1996,1100056,527


[1] 154730      4

In [8]:
# Exportando os dados (Nascimentos_muni = UF, Município, Ano e Nascimentos)
write.table(dataframe, 'Nascimentos_muni.csv', sep = ";", dec = ".", row.names = FALSE)

In [17]:
##raca/cor
estados <- c('RO','AC','AM','RR','PA','AP','TO','MA','PI','CE','RN','PB','PE','AL','SE','BA','MG','ES','RJ','SP','PR','SC','RS','MS','MT','GO','DF')
dataframe <- data.frame()

endpoint <- paste0(url_base,"/","sql_query")

for (estado in estados){
    params = paste0('{
          "token": {
            "token": "',token,'"
          },
          "sql": {
            "sql": {"query":"SELECT res_SIGLA_UF, CODMUNNASC, ano_nasc, RACACOR, COUNT(1) FROM \\"datasus-sinasc\\" WHERE res_SIGLA_UF = \'',estado,'\' GROUP BY res_SIGLA_UF, CODMUNNASC, ano_nasc, RACACOR", "fetch_size": 65000}
          }
          
        }')
    
    request <- POST(url = endpoint, body = params, encode = "form")
    df_premat <- convertRequestToDF(request)
    names(df_premat) <- c('UF', 'Municipio', 'Ano', 'Raca', 'Nascidos')
    dataframe <- rbind(dataframe, df_premat)
    
    repeat {
        
        cursor <- content(request)$cursor
        
        params = paste0('{
          "token": {
            "token": "',token,'"
          },
          "sql": {
            "sql": {"query":"SELECT res_SIGLA_UF, CODMUNNASC, ano_nasc, RACACOR, COUNT(1) FROM \\"datasus-sinasc\\" WHERE res_SIGLA_UF = \'',estado,'\' GROUP BY res_SIGLA_UF, CODMUNNASC, ano_nasc, RACACOR", "fetch_size": 65000, "cursor": "',cursor,'"}
          }
        }')


        request <- POST(url = endpoint, body = params, encode = "form")
        
        if (length(content(request)$rows) == 0)
            break
        else print("oi")
        
        df_premat <- convertRequestToDF(request)
        names(df_premat) <- c('UF', 'Municipio', 'Ano', 'Raca', 'Nascidos')
        dataframe <- rbind(dataframe, df_premat)
    }
}

ERROR: Error: $ operator is invalid for atomic vectors


In [11]:
head(dataframe)
dim(dataframe)

,UF,Municipio,Ano,Raca,Nascidos
,<fct>,<fct>,<fct>,<fct>,<fct>
1,RO,110000,2014,4,1
2,RO,1100000,1996,9,1
3,RO,1100000,1997,4,1
4,RO,1100000,1998,1,1
5,RO,1100000,1998,4,3
6,RO,1100000,1998,9,3


[1] 421255      5

In [12]:
# Exportando os dados (Nascimentos_muni = UF, Município, Ano e Nascimentos)
write.table(dataframe, 'raca_muni.csv', sep = ";", dec = ".", row.names = FALSE)